In [29]:
import numpy as np
import pandas as pd
import re
import glob
import os 
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [6]:
path_train="scores/stack_train/"
path_test="scores/stack_test/"

In [7]:
dict_train = {} 
dict_test = {}
train_files = [i for i in os.listdir(path_train)]
test_files = [i for i in os.listdir(path_test)]
for f in train_files:
    dict_train[re.sub("\.csv","",f)] = pd.read_csv(path_train+f)
for f in test_files:
    dict_test[re.sub("\.csv","",f)] = pd.read_csv(path_test+f)

In [10]:
col_no_ID=['class1', 'class2', 'class3', 'class4', 'class5', 'class6', 'class7',
       'class8', 'class9']
del dict_train[".DS_Store"]
del dict_test[".DS_Store"]

In [11]:
#Affecting  columns name
for name in dict_test.keys():
    dict_train[name].columns=[name+"_"+s for s in col_no_ID]+["ID"]
    dict_test[name].columns=[name+"_"+s for s in col_no_ID]+["ID"]

In [12]:
stack_train=dict_train["nowdw_XGB_medium"].merge(dict_train["nowdw_XGB_small"],
on="ID").merge(dict_train["nowdw_XGB_tall"],on="ID").merge(dict_train["XGB_M_d2v"],
on="ID").merge(dict_train["XGB_S_d2v"],on="ID").merge(dict_train["XGB_T_d2v"],
on="ID").merge(dict_train["XGB_S_tfidf"],on="ID").merge(dict_train["XGB_M_tfidf"],
on="ID").merge(dict_train["XGB_T_tfidf"],on="ID").merge(dict_train["XGB_S_w2v"],
on="ID").merge(dict_train["XGB_M_w2v"],on="ID").merge(dict_train["XGB_T_w2v"],
on="ID").merge(dict_train["LGBM_S_d2v"],on="ID").merge(dict_train["LGBM_M_d2v"],
on="ID").merge(dict_train["LGBM_T_d2v"],on="ID").merge(dict_train["LGBM_S_tfidf"],
on="ID").merge(dict_train["LGBM_M_tfidf"],on="ID").merge(dict_train["LGBM_T_tfidf"],
on="ID").merge(dict_train["LGBM_S_w2v"],on="ID").merge(dict_train["LGBM_M_w2v"],
on="ID").merge(dict_train["LGBM_T_w2v"],on="ID").merge(dict_train["adaboost_d2v"],
on="ID").merge(dict_train["adaboost_tfidf"],on="ID").merge(dict_train["adaboost_w2v"],
on="ID").merge(dict_train["logit_d2v"],on="ID").merge(dict_train["logit_tfidf"],
on="ID").merge(dict_train["logit_w2v"],on="ID")#.merge(dict_train["nowdw_neural_net"],on="ID")
#a tester 

In [13]:
stack_test=dict_test["nowdw_XGB_medium"].merge(dict_test["nowdw_XGB_small"],
on="ID").merge(dict_test["nowdw_XGB_tall"],on="ID").merge(dict_test["XGB_M_d2v"],
on="ID").merge(dict_test["XGB_S_d2v"],on="ID").merge(dict_test["XGB_T_d2v"],
on="ID").merge(dict_test["XGB_S_tfidf"],on="ID").merge(dict_test["XGB_M_tfidf"],
on="ID").merge(dict_test["XGB_T_tfidf"],on="ID").merge(dict_test["XGB_S_w2v"],
on="ID").merge(dict_test["XGB_M_w2v"],on="ID").merge(dict_test["XGB_T_w2v"],
on="ID").merge(dict_test["LGBM_S_d2v"],on="ID").merge(dict_test["LGBM_M_d2v"],
on="ID").merge(dict_test["LGBM_T_d2v"],on="ID").merge(dict_test["LGBM_S_tfidf"],
on="ID").merge(dict_test["LGBM_M_tfidf"],on="ID").merge(dict_test["LGBM_T_tfidf"],
on="ID").merge(dict_test["LGBM_S_w2v"],on="ID").merge(dict_test["LGBM_M_w2v"],
on="ID").merge(dict_test["LGBM_T_w2v"],on="ID").merge(dict_test["adaboost_d2v"],
on="ID").merge(dict_test["adaboost_tfidf"],on="ID").merge(dict_test["adaboost_w2v"],
on="ID").merge(dict_test["logit_d2v"],on="ID").merge(dict_test["logit_tfidf"],
on="ID").merge(dict_test["logit_w2v"],on="ID")#.merge(dict_test["nowdw_neural_net"],on="ID")

In [14]:
#quand le merging est fini : table unique, on peut monter un meta_model

In [15]:
y=pd.read_csv("bases/training_variants").Class.values - 1

In [16]:
ID_test=stack_test["ID"]

In [17]:
X_train=np.array(stack_train.drop("ID",axis=1))

In [18]:
X_test=np.array(stack_test.drop("ID",axis=1))

In [19]:
param_test1 = {
 'max_depth':[1,3,5],
 'min_child_weight':[1,3,5]
}


In [25]:
#Testing other meta_model

In [39]:
cv_xgb=cross_val_score(XGBClassifier(objective="multi:softprob",
seed=26),X_train,y,scoring="neg_log_loss",cv=10)

In [37]:
cv_log=cross_val_score(LogisticRegression(),X_train,y,scoring="neg_log_loss",cv=10)

In [38]:
cv_log.mean()

-0.90562516735978016

In [41]:
clf = LogisticRegression()
clf.fit(X_train,y)
y_test=clf.predict_proba(X_test)

classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
sub = pd.DataFrame(y_test, columns=classes)
sub['ID'] = ID_test
sub.to_csv("scores/stacking_linear.csv",index=False)